**BERT 모델 불러오기**

In [1]:
from transformers import BertModel, BertTokenizer
import torch

# 1. BERT 토크나이저와 모델을 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
# tokenizer의 내장함수를 이용해 특수 토큰을 불러오는 함수를 작성해주세요
# UNK 토큰
unk_token = tokenizer.unk_token
print(f"UNK Token: {unk_token}")

# SEP 토큰
sep_token = tokenizer.sep_token
print(f"SEP Token: {sep_token}")

# PAD 토큰
pad_token = tokenizer.pad_token
print(f"PAD Token: {pad_token}")

# CLS 토큰
cls_token = tokenizer.cls_token
print(f"CLS Token: {cls_token}")

# MASK 토큰
mask_token = tokenizer.mask_token
print(f"MASK Token: {mask_token}")

UNK Token: [UNK]
SEP Token: [SEP]
PAD Token: [PAD]
CLS Token: [CLS]
MASK Token: [MASK]


**1.BERT Tokenizer의 특수 토큰**

아래 코드에서 특수 토큰을 불러오는 함수를 작성하고 각 토큰이 무엇을 의미하는지 적어주세요

- [UNK] : 알 수 없는 토큰을 나타냄
- [SEP] : 문장의 구분을 나타냄
- [PAD] : 패딩을 나타냄
- [CLS] : 문장의 시작을 나타냄
- [MASK] : 마스킹을 나타냄

**tokenizer를 이용해 문장을 토큰화**

토큰화를 통해 문자를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 한다.

In [7]:
# 입력 문장
sentence = "Hello, this is a sentence for tokenization."

# 문장 토큰화 및
inputs = tokenizer(sentence, return_tensors="pt")

# 토큰 ID를 다시 토큰으로 변환
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())  # 텐서를 리스트로 변환

print("Token IDs:", inputs['input_ids'])
print("Tokens:", tokens)

Token IDs: tensor([[  101,  7592,  1010,  2023,  2003,  1037,  6251,  2005, 19204,  3989,
          1012,   102]])
Tokens: ['[CLS]', 'hello', ',', 'this', 'is', 'a', 'sentence', 'for', 'token', '##ization', '.', '[SEP]']


**2. 토큰화**
토큰화 이후에 [CLS] 토큰과 [SEP] 토큰이 추가된 이유를 작성하세요

- [CLS] 토큰이 추가된 이유 : 문장분류같은 작업에서 [cls] 토큰의 출력을 사용해 문장의 최종 의미를 추출함. 감정분석, 문장 관계 추론 등의 작업에서 [cls]토큰이 중요한 역할을 함
- [SEP] 토큰이 추가된 이유 : Bert모델에서는 두 문장의 입력으로 받아 문장 관계를 학습하는 경우가 많기 때문에, 두 문장을 구분할 필요가 있음

**감성 분류를 위한 사전 학습된 BERT 모델을 이용해 감성분류하기**

In [8]:
import torch
from transformers import BertForSequenceClassification
from torch.nn.functional import softmax

# 감성 분류를 위한 사전 학습된 모델
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = BertForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [9]:
# sentence에 대해 BERT를 이용해 감성분류 하는 함수
def sentiment_analysis(sentence):

    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

    with torch.no_grad():
      outputs = model(**inputs)

    logits = outputs.logits
    probs = softmax(logits, dim=-1)

    probs_list = probs[0].tolist()

    # 클래스 레이블 정의
    class_label = ['매우 부정적', '부정적', '중립적', '긍정적', '매우 긍정적']

    # 결과 출력 (소수점 2자리로 출력)
    for i, prob in enumerate(probs_list):
        print(f"{class_label[i]}일 확률: {prob:.2f}")

**3. BERT를 이용하여 각각 class에 맞는 sentence 작성해보기**

1. 매우 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
2. 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
3. 중립적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
4. 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
5. 매우 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요

In [18]:
# 매우 부정적인 문장 작성
sentence = "수업 너무 지루해"

# 감성분석 수행
sentiment_analysis(sentence)


매우 부정적일 확률: 0.19
부정적일 확률: 0.21
중립적일 확률: 0.31
긍정적일 확률: 0.17
매우 긍정적일 확률: 0.13


In [11]:
# 부정적인 문장 작성
sentence = "아이스크림 맛없어"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.22
부정적일 확률: 0.31
중립적일 확률: 0.32
긍정적일 확률: 0.09
매우 긍정적일 확률: 0.06


In [16]:
# 중립적 문장 작성
sentence = "날씨가 그냥그래"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.15
부정적일 확률: 0.18
중립적일 확률: 0.32
긍정적일 확률: 0.23
매우 긍정적일 확률: 0.13


In [17]:
# 긍정적 문장 작성
sentence = "날씨가 좋아"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.15
부정적일 확률: 0.18
중립적일 확률: 0.32
긍정적일 확률: 0.23
매우 긍정적일 확률: 0.13


In [19]:
# 매우 긍정적인 문장 작성
sentence = "정말정말 좋다"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.02
부정적일 확률: 0.02
중립적일 확률: 0.09
긍정적일 확률: 0.20
매우 긍정적일 확률: 0.67


**4. BERT의 감성 분류 기준에 대해 생각을 적어보기**


BERT가 어떤 요소에 따라 감성 분류 기준을 나누는지에 대해 적어보세요

내가 생각하는 분류 기준 : 문장의 앞뒤 관계를 학습해서 하는거 같고, 부정어의 사용에 따라 나뉘는거같다.